# Funções

In [24]:
import networkx,numpy as np,cmath
np.set_printoptions(precision=5)
class Trafo:
    def __init__(self,vp,vs,s,xf) -> None:
        self.vp = vp
        self.vs = vs
        self.s = s
        self.xf = xf
        self.relacao = vp/vs

class Gerador:
    def __init__(self,s,v,xd):
        self.s = s
        self.v = v
        self.xd = xd


def polar(number:complex):
    module = abs(number)
    angle = np.rad2deg(np.arctan2(number.imag,number.real))

    return module,angle

def retangular(number:tuple[float,float]):
    return cmath.rect(number[0],np.deg2rad(number[1]))

def calcula_corrente(potencia_base,tensao_base):
    return potencia_base/(3**.5 * tensao_base)

def angulo_fp(fp,indutivo = True):
    return np.rad2deg(np.arccos(fp)) * (-1 if indutivo else 1)

def calcula_z_pu(potencia_base,trafo:Trafo,tensao_base_primario):
    return trafo.xf*potencia_base/trafo.s * (trafo.vp/tensao_base_primario)**2*1j

def calcula_tensao_base(base_anterior,trafo:Trafo,lado_contrario=False):
    return base_anterior/trafo.relacao if not lado_contrario else base_anterior * trafo.relacao

def cria_matriz_admitancia(nodes:list[int],edges):
    # Create a directed graph
    G = networkx.DiGraph()
    G.add_nodes_from(nodes)
    for u, v, w in edges:
        G.add_weighted_edges_from([(u, v, -w), (v, u, -w)])
    # Display the adjacency matrix
    adj_matrix = networkx.adjacency_matrix(G).todense()
    for i in G.nodes():
        adj_matrix[i - 1, i - 1] = -sum(G[i][j]["weight"] for j in G[i])

    return adj_matrix

def calcula_z_base(tensao_base_antiga,tensao_base_nova,potencia_base_antiga,potencia_base_nova,reatancia):
    '''
    Alteração de Z para nova base em p.u
    z' = z*nova_base/antiga_base*(vb_antiga/vb_nova)**2
    '''
    zb = 1j*reatancia*(potencia_base_nova/potencia_base_antiga)*(tensao_base_antiga/tensao_base_nova)**2
    return zb

def potencia_ativa(tensao_terminal,tensao_gerador,reatancia,angulo_gerador):
    return abs(tensao_terminal)*abs(tensao_gerador)/abs(reatancia) * np.sin(np.deg2rad(angulo_gerador))

def potencia_reativa(tensao_terminal,tensao_gerador,reatancia,angulo_gerador):
    return abs(tensao_terminal)/abs(reatancia) * (abs(tensao_gerador)*np.cos(np.deg2rad(angulo_gerador))-abs(tensao_terminal))

def novo_angulo_excitacao(tensao_gerador,angulo_gerador,diferenca):

    # e1 sen1 = e2 sen2
    sen2 = abs(tensao_gerador)*np.sin(np.deg2rad(angulo_gerador))/((1+diferenca)*abs(tensao_gerador))
    gama2 = np.rad2deg(np.arcsin(sen2))
    return gama2


# 1)

In [25]:
ta = Trafo(13.8e3,138e3,150e6,7/100)
tb = Trafo(138e3,13.8e3,50e6,8/100)
tc = Trafo(138e3,13.8e3,50e6,9/100)
g1 = Gerador(100e6,13.8e3,20/100)
lta = 2.6+8j
ltb = 1.85+5j

In [26]:
vb5 = vb4 = 34.5e3
sb = 50e6
vb3 = vb2 = calcula_tensao_base(vb4,tb,lado_contrario=True)
vb3

345000.0

In [27]:
vb1 = calcula_tensao_base(vb2,ta,lado_contrario=True)
vb1

34500.0

In [28]:
zta = calcula_z_pu(sb,ta,vb1)
zta

0.0037333333333333346j

In [29]:
ztb = calcula_z_pu(sb,tb,vb3)
ztb

0.012800000000000002j

In [30]:
ztc = calcula_z_pu(sb,tc,vb2)
ztc

0.014400000000000003j

In [31]:
#gerador
zg = calcula_z_base(g1.v,vb1,g1.s,sb,g1.xd)
zg

0.016000000000000004j

In [32]:
#admitancias
yta = 1/zta
ytb = 1/ztb
ytc = 1/ztc
ylta = 1/lta
yltb = 1/ltb
yg = 1/zg

In [33]:
conexoes = [(1,2,yta),(2,3,ylta),(2,5,ytc),(3,4,ytb),(1,1,yg),(4,5,yltb)]

matriz_y = cria_matriz_admitancia(range(1,6),conexoes)
matriz_y

array([[-0.     -3.30357e+02j,  0.     +2.67857e+02j,
         0.     +0.00000e+00j,  0.     +0.00000e+00j,
         0.     +0.00000e+00j],
       [ 0.     +2.67857e+02j,  0.03674-3.37415e+02j,
        -0.03674+1.13058e-01j,  0.     +0.00000e+00j,
         0.     +6.94444e+01j],
       [ 0.     +0.00000e+00j, -0.03674+1.13058e-01j,
         0.03674-7.82381e+01j,  0.     +7.81250e+01j,
         0.     +0.00000e+00j],
       [ 0.     +0.00000e+00j,  0.     +0.00000e+00j,
         0.     +7.81250e+01j,  0.06509-7.83009e+01j,
        -0.06509+1.75917e-01j],
       [ 0.     +0.00000e+00j,  0.     +6.94444e+01j,
         0.     +0.00000e+00j, -0.06509+1.75917e-01j,
         0.06509-6.96204e+01j]])

# 2)

In [34]:
sb = 200e6
vb = 345e3

n12 = 138/34.5 #Relação entre o primário e secundário

# Passando para a base 200MVA e 345kV
zps = calcula_z_base(138e3,vb,33e6,sb,9/100)
zpt = calcula_z_base(138e3,vb,33e6,sb,7/100)
zst = calcula_z_base(34.5e3,vb/n12,20e6,20e6,12/100)
zps,zpt,zst

(0.08727272727272728j, 0.0678787878787879j, 0.019200000000000002j)

In [35]:
zp = (zps+zpt-zst)/2
zs = (zps+zst-zpt)/2
zt = (zpt+zst-zps)/2
zp,zs,zt

(0.06797575757575759j, 0.019296969696969687j, -9.69696969696926e-05j)

# 3)

In [36]:
xf = .05j # propria base. p.u
zeq = xf
yeq = 1/zeq
yeq

-20j

## a)

In [37]:
dv = 0.03
c = 1/(1+dv)
c

0.970873786407767

In [38]:
y11 = yeq
y12 = y21 = -c*yeq
y22 = abs(c)**2 * yeq

matriz_y = [[y11,y12],[y21,y22]]
for l in matriz_y:
    print(l)

[-20j, 19.41747572815534j]
[19.41747572815534j, -18.851918182675085j]


## b)

In [39]:
angulo = 1
c2 = retangular((1,1)) # abc' é 1 grau menor. Portanto "ira aumentar -1 grau", ou seja alpha = -1 e c = 1|_-alpha = 1|_1
c2

(0.9998476951563913+0.01745240643728351j)

In [40]:
y11 = yeq
y12 = y21 = -c2*yeq
y22 = abs(c2)**2 * yeq

matriz_y = [[y11,y12],[y21,y22]]
for l in matriz_y:
    print(l)

[-20j, (-0.34904812874567026+19.996953903127825j)]
[(-0.34904812874567026+19.996953903127825j), -20j]


# 4)

In [41]:
sb = 100e6
vb = 13.8e3
xd = 1j #pu
ig = 3300 #modulo
fp = .8 #indutivo

In [42]:
i_base = calcula_corrente(sb,vb)
i_base

4183.697602823375

In [43]:
vt_pu = 1
i_pu = retangular((ig/i_base,angulo_fp(fp)))
polar(i_pu)

(0.7887759377668667, -36.86989764584401)

In [44]:
ei_pu = xd*i_pu + vt_pu
polar(ei_pu)

(1.602716008942395, 23.186144413469076)

In [45]:
gama = polar(ei_pu)[1]
p = potencia_ativa(vt_pu,ei_pu,xd,gama)
q = potencia_reativa(vt_pu,ei_pu,xd,gama)
print(p)
print(q)

0.6310207502134934
0.47326556266011965


In [46]:
gama2 = novo_angulo_excitacao(ei_pu,gama,-10/100)
q2 = potencia_reativa(vt_pu,ei_pu*.9,xd,gama2)
print(gama2)
print(q2)

25.942329517877997
0.29709625051859345
